In [41]:
import pandas as pd
import os

In [42]:
df_all_trials = pd.read_csv("./exported_csv/fixation_counts/IVT/summary_all_trials.csv")
df_aoi1  = pd.read_csv("./exported_csv/fixation_counts/IVT_triangle/summary_aoi1.csv")

In [43]:
# === exp_idとtrialで結合 ===
merged_df = pd.merge(
    df_all_trials,
    df_aoi1,
    on=['id', 'exp_id', 'trial'],
    how='inner',
    suffixes=('_all', '_aoi1')
)

# === 列名を最初に統一 ===
merged_df = merged_df.rename(columns={
    'aoi': 'triangle',
    'outside_aoi1': 'triangle_aoi_outside'
})

In [44]:
# === データ型確認
print("変換前データ型")
print(merged_df.dtypes)

# === 数値に変換する列を指定 ===
cols_to_numeric = [
    'total',
    'left_eye',
    'right_eye',
    'nose',
    'mouth',
    'triangle',
    'triangle_aoi_outside',
    'outside_all'  # ←追加
]

変換前データ型
id                        int64
exp_id                    int64
trial                     int64
left_eye                 object
right_eye                object
nose                     object
mouth                    object
outside_all              object
triangle_aoi_outside     object
triangle                float64
total                   float64
dtype: object


In [45]:
# 数値に変換（エラーはNaNに）
for col in cols_to_numeric:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    # === データ型確認
print("\n変換後データ型")
print(merged_df.dtypes)


変換後データ型
id                        int64
exp_id                    int64
trial                     int64
left_eye                float64
right_eye               float64
nose                    float64
mouth                   float64
outside_all             float64
triangle_aoi_outside    float64
triangle                float64
total                   float64
dtype: object


In [46]:
# === parts_aoi_outsideの計算 ===
merged_df['parts_aoi_outside'] = (
    merged_df['total'] -
    (
        merged_df['left_eye'] +
        merged_df['right_eye'] +
        merged_df['nose'] +
        merged_df['mouth']
    )
)


In [47]:
# === 出力列を並べる ===
output_columns = [
    'id',
    'exp_id',
    'trial',
    'total',
    'left_eye',
    'right_eye',
    'nose',
    'mouth',
    'parts_aoi_outside',
    'triangle',
    'triangle_aoi_outside'
]

# === 出力用DataFrame
final_df = merged_df[output_columns]

In [ ]:
# === 各行単位でファイルを出力 ===
for _, row in final_df.iterrows():
    exp_id = row['exp_id']
    trial = row['trial']

    # 出力用DataFrame（行単位）
    row_df = row.to_frame().T  # Series→DataFrameにする

    
    # 出力ファイル名
    filename = f"exported_csv/fixation_counts/IVT_image_group/merged_exp{int(exp_id:3)}_trial{int(trial:3)}.csv"


    # 1ファイルずつ追記
    if exp_id == 1 and trial == 0:
        # 1回目はヘッダーあり
        row_df.to_csv(filename, mode="w", index=False, header=True)
    else:
        # 2回目以降はヘッダーなし
        row_df.to_csv(filename, mode="a", index=False, header=False)

    # CSV保存
    # row_df.to_csv(filename, index=False)

    # print(f"✅ 出力: {filename}")
